In [0]:
import pandas as pd
#from google.colab import files

In [0]:
from keras.layers import Conv2D, Activation, BatchNormalization, SeparableConv2D,MaxPooling2D,GlobalAveragePooling2D
from keras import Model
from keras import layers
from keras.regularizers import l2 
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
import pickle
from keras.utils import np_utils
from keras.layers import Input

In [0]:
def mini_XCEPTION(input_shape, num_classes, l2_regularization=0.01):
  regularization = l2(l2_regularization)
  
  img_input = Input(input_shape)
  x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization,use_bias=False)(img_input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization,use_bias=False)(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  
  residual = Conv2D(16, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
  residual = BatchNormalization()(residual)
  x = SeparableConv2D(16, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = SeparableConv2D(16, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
  x = layers.add([x, residual]) 

  residual = Conv2D(32, (1, 1), strides=(2, 2),padding='same', use_bias=False)(x) 
  residual = BatchNormalization()(residual)
  x = SeparableConv2D(32, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = SeparableConv2D(32, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
  x = layers.add([x, residual]) 

  residual = Conv2D(64, (1, 1), strides=(2, 2),padding='same', use_bias=False)(x)
  residual = BatchNormalization()(residual)
  x = SeparableConv2D(64, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = SeparableConv2D(64, (3, 3), padding='same', kernel_regularizer=regularization,use_bias=False)(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
  x = layers.add([x, residual])

  residual = Conv2D(128, (1, 1), strides=(2, 2),padding='same', use_bias=False)(x)
  residual = BatchNormalization()(residual)

  x = SeparableConv2D(128, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = SeparableConv2D(128, (3, 3), padding='same', kernel_regularizer=regularization,use_bias=False)(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
  x = layers.add([x, residual])
  x = Conv2D(num_classes, (3, 3),
             #kernel_regularizer=regularization, 
             padding='same')(x)
  x = GlobalAveragePooling2D()(x)

  output = Activation('softmax',name='predictions')(x)
  model = Model(img_input, output)
  return model  

In [0]:
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

In [0]:
import io
data = pd.read_csv('fer2013.csv')

In [0]:
import numpy as  np

train = data[data.Usage == "Training"]
test = data[data.Usage == "PrivateTest"]
test_pub = data[data.Usage == "PublicTest"]


train_labels = train["emotion"].values
test_labels = test["emotion"].values
test_pub_labels = test_pub["emotion"].values

train_ds = train["pixels"].values
test_ds = test["pixels"].values
test_pub_ds = test_pub["pixels"].values

train_ds_list = list()
test_ds_list = list()
test_pub_ds_list = list()

for i in range(28709):
  train_ds_list.append(np.array(train_ds[i].split(" ")))
  train_ds_list[i] = train_ds_list[i].astype(np.float)

for i in range(3589):
  test_ds_list.append(np.array(test_ds[i].split(" ")))
  test_ds_list[i] = test_ds_list[i].astype(np.float)

for i in range(3589):
  test_pub_ds_list.append(np.array(test_pub_ds[i].split(" ")))
  test_pub_ds_list[i] = test_pub_ds_list[i].astype(np.float)

train_np = np.array(train_ds_list)
test_np = np.array(test_ds_list)
test_pub_np = np.array(test_pub_ds_list)



train_np = train_np.reshape(train_np.shape[0], 48, 48, 1)
test_np = test_np.reshape(test_np.shape[0],48, 48, 1)
test_pub_np = test_pub_np.reshape(test_pub_np.shape[0], 48, 48, 1)

train_labels = np_utils.to_categorical(train_labels, 7)
test_labels = np_utils.to_categorical(test_labels, 7)
test_pub_labels = np_utils.to_categorical(test_pub_labels, 7) 

patience = 50
batch_size = 64
num_epochs = 60


train_big = np.vstack([train_np, test_pub_np])
train_label_big = np.concatenate([train_labels, test_pub_labels])

In [0]:
early_stop = EarlyStopping('val_loss', patience=patience)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1,
                                  patience=int(patience/4), verbose=1)
model_names ='weights_mini_xception.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,
                                                    save_best_only=True)
callbacks = [model_checkpoint, early_stop, reduce_lr]


In [43]:
model = mini_XCEPTION((48, 48, 1), 7)
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [45]:
model.fit_generator(data_generator.flow(train_big, train_label_big,
                                            batch_size),
                        steps_per_epoch=len(train_big) / batch_size,
                        epochs=num_epochs, verbose=1, callbacks=callbacks,
                        validation_data=[test_np,test_labels])
model.save_weights("weights_60_epochs_xception.hdf5")


Epoch 1/60
505/504 [==============================] - 29s 57ms/step - loss: 1.7791 - acc: 0.3264 - val_loss: 1.6779 - val_acc: 0.3720

Epoch 00001: val_loss improved from inf to 1.67790, saving model to weights_mini_xception.01-0.37.hdf5
Epoch 2/60
505/504 [==============================] - 22s 43ms/step - loss: 1.5394 - acc: 0.4246 - val_loss: 1.5032 - val_acc: 0.4372

Epoch 00002: val_loss improved from 1.67790 to 1.50322, saving model to weights_mini_xception.02-0.44.hdf5
Epoch 3/60
505/504 [==============================] - 22s 44ms/step - loss: 1.4187 - acc: 0.4710 - val_loss: 1.4955 - val_acc: 0.4820

Epoch 00003: val_loss improved from 1.50322 to 1.49547, saving model to weights_mini_xception.03-0.48.hdf5
Epoch 4/60
505/504 [==============================] - 22s 43ms/step - loss: 1.3306 - acc: 0.5008 - val_loss: 1.4028 - val_acc: 0.4879

Epoch 00004: val_loss improved from 1.49547 to 1.40278, saving model to weights_mini_xception.04-0.49.hdf5
Epoch 5/60
505/504 [================